#### imports

In [5]:
import pandas as pd
import networkx as nx
import numpy as np
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt
import matplotlib as mpl
import itertools
from copy import deepcopy
from scipy.integrate import solve_ivp
from itertools import combinations_with_replacement
from emn_model import (
    get_clean_dataset, generate_bipartite_network, diffusion_dynamics, tree_project_network
)

In [2]:


TEX_PLOTS = True

# Plotting config
if TEX_PLOTS:
    plt.rc("text", usetex=True)
    plt.rc("font", family="serif")
    plt.rc("text.latex", preamble=r"""
         \usepackage{times}
         \usepackage{mathptmx}
         \usepackage{siunitx}""")
else:
    plt.rc("text", usetex=False)

mpl.rcParams["figure.dpi"] = 200
mpl.rcParams["legend.fontsize"] = 14
mpl.rcParams["font.size"] = 14
mpl.rcParams["axes.labelsize"] = mpl.rcParams["axes.titlesize"] = 16
mpl.rcParams["axes.grid"] = True
mpl.rcParams["grid.alpha"] = 0.2
mpl.rcParams["axes.axisbelow"] = True
mpl.rcParams["lines.linewidth"] = 1

cmap = mpl.cm.get_cmap("Dark2")
mpl.rcParams["axes.prop_cycle"] = mpl.cycler(color=cmap.colors)

np.random.seed(1231)

# Background

### Tree classification

I have removed the cohorts and group them by diameter. The groups are based on their diameter, which signifies their relative age-group. 

* Saplings (young trees): 0 $<$ d $\le$ 15, diameter sizes
* Maturing trees: 15$<$ d $\le$ 35, diameter sizes
* Old trees: 35$<$ d, diameter sizes 

It the current data-set the diameters range from $0.56$ to $53$ diameter sizes. It might be a good idea to sample between $1$ and $55$ (but whatever works). 

### Macro-statistics

The main interest of our model is to what extent older trees (with more resources) are capable of facilitating the success of young trees in terms of carbon transfer. I have thought of a few macro-statistics, but please feel free to add more.  

#### Sapling Succes 

We do not have mortality in our model, so we have to define succes ourselves.

Considering the sapling group, a sapling is succesful when: 
* Succesful: diameter grew at least 2x in size. Above or equal $200\%$ growth. 
* Unsuccesful: diameter grew less than 2x in size. Below $200\%$ growth. 

(below I will add the code to extract these dynamics)


#### Sapling Competition

In our model some saplings are way better growers than others, regardless of size. This is potentially due to its connections. Hence, a measure that resembles this competition will be nice.

Measure of competition (Among samplings): 
* Standard deviation of the growth percentage. 


#### Nutrient diffusion
?? any ideas? Something about how well and far nutrients can diffuse? 

# Results

#### Initialization of the network.

In [3]:
df = get_clean_dataset()
B = generate_bipartite_network(df)
G = tree_project_network(B)

In [4]:
G.nodes

NodeView((2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 30, 31, 35, 40, 41, 44, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 62, 64, 69, 74, 75, 81, 83, 85, 86, 87, 88, 89, 90, 91))